In [158]:
import numpy as np


def create_random_config(N):
    """Create spin configuration of NxN lattice"""
    return 2 * np.random.randint(2, size=(N, N)) - 1

# better way:
# https://tanyaschlusser.github.io/posts/mcmc-and-the-ising-model/
def get_dH(lattice, trial_location):
    """ H = - Sum_<ij>(s_i s_j) """
    i, j = trial_location
    height, width = lattice.shape
    H, Hflip = 0, 0
    for di, dj in ((-1, 0), (1, 0), (0, -1), (0, 1)):
        ii = (i + di) % height
        jj = (j + dj) % width
        H -= lattice[ii, jj] * lattice[i, j]
        Hflip += lattice[ii, jj] * lattice[i, j]
    return Hflip - H


def neighbours_sum(lattice, i, j):
    (N, _) = lattice.shape
    up = (i, (j-1)%N)
    left = ((i-1)%N, j)
    right = ((i+1)%N, j)
    down = (i, (j+1)%N)
    s = lattice[up] + lattice[down] + lattice[left] + lattice[right]
    return  s

def step(lattice, beta):
    # Pick a random position in a lattice
    (N, _) = lattice.shape
    indices = np.random.randint(0, high=N, size=2)
    i, j = indices[0], indices[1]
    entry = lattice[i, j]
    n = neighbours_sum(lattice, i, j)
    return entry
    
# # Request two random integers between 0 and 3 (exclusive)
# indices =  np.random.randint(0, high=3, size=2)

# # Extract the row and column indices
# i = indices[0]
# j = indices[1]

# # Put 2 at the random position
# grid[i,j] = 2   

config = create_random_config(N=10)
print(config)


[[ 1 -1 -1  1 -1  1 -1 -1  1  1]
 [-1 -1  1 -1  1  1  1  1  1  1]
 [-1 -1 -1  1  1  1 -1  1 -1  1]
 [ 1  1 -1  1 -1  1  1 -1 -1  1]
 [ 1  1  1 -1  1  1 -1  1  1 -1]
 [ 1 -1  1 -1 -1 -1 -1  1  1  1]
 [ 1  1 -1  1 -1 -1 -1  1  1 -1]
 [ 1  1  1 -1 -1  1  1  1  1  1]
 [ 1  1 -1 -1  1  1  1 -1  1  1]
 [-1  1 -1  1 -1 -1 -1 -1  1  1]]


In [159]:
print(neighbours_sum(config, 3, 3))

-2


In [143]:
a[(0,0)]

9